<a href="https://colab.research.google.com/github/WizardOfCodes442/student-facial-biometric-project/blob/main/testingcnnwithobamapic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import requests
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import layers, models
from sklearn.metrics.pairwise import cosine_similarity

# Load the MobileNetV2 model
def load_model():
    base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='softmax')
    ])
    return model

# Download an image from a URL
def download_image(url):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        image = np.asarray(bytearray(response.raw.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        return image
    else:
        print(f"Error downloading image from URL: {url}")
        return None

# Preprocess image for the model
def preprocess_image(image):
    resized_image = cv2.resize(image, (224, 224))
    processed_image = preprocess_input(resized_image)
    return processed_image

# Extract face encoding from the image
def extract_face_encoding(model, image):
    processed_image = preprocess_image(image)
    processed_image = np.expand_dims(processed_image, axis=0)  # Add batch dimension
    encoding = model.predict(processed_image)
    return encoding.flatten()

# Load the face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Detect and extract face from an image
def detect_face(image):
    if image is None:
        return None
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_image, 1.1, 4)

    if len(faces) == 0:
        return None

    x, y, w, h = faces[0]  # Take the first detected face
    return image[y:y+h, x:x+w]  # Return cropped face

# URLs for 3 training images and 1 verification image (replace with your actual URLs)
train_image_urls = [
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRxJLWdWHgf6_XGwYB1tTfJvxZ9AHDA2sElJaF_7cvPm4BzsTlKDGOqYyc9S7FJBX-p-fU&usqp=CAU",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTCcgE2H9WH7xph_Y2xrr85ltidewM9gcWknw&s",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRb4J98Mji_p2b65SRtlxg6tOYXhp3dBX2v8A&s",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS9lXHJHGcWiBbJ_K5DON4raTwnowEoz2RDvSze8waMy6xwmBJ0DBTAlsViz7e5ooZshzM&usqp=CAU",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTn6l_iqbERyP8ccI2w0XbcdH00b1ik6I8GMbb0vS0E4_xCw_a_HqEBWgIeQ2oOzXJibHw&usqp=CAU",
    "https://politico.eu/cdn-cgi/image/width=1160,height=703,fit=crop,quality=80,onerror=redirect,format=auto/wp-content/uploads/2020/05/GettyImages-1225292516.jpg",
    "https://media.licdn.com/dms/image/C4E12AQFLaKy8cAAxvQ/article-cover_image-/0/1520103496443?e=2147483647&v=beta&t=AXoewqNsSTqdFHnUjGYId2WXnbX8wfaw9n_wcihrAkI"
]
verify_image_url = "https://i.ytimg.com/vi/BdjoHA5ocwU/hq720.jpg?sqp=-oaymwEhCK4FEIIDSFryq4qpAxMIARUAAAAAGAElAADIQj0AgKJD&rs=AOn4CLBzFCX11mQ0tmO692NyJopkOnlT5Q"
# Load the MobileNetV2 model
model = load_model()

# Load and process the training images
train_encodings = []
for url in tshrink_600_2000rain_image_urls:
    train_image = download_image(url)

    if train_image is None:
        print(f"Failed to download or process training image from {url}")
        continue

    train_face = detect_face(train_image)

    if train_face is not None:
        train_encoding = extract_face_encoding(model, train_face)
        train_encodings.append(train_encoding)
    else:
        print(f"No face detected in training image from {url}")

# Average the encodings from the 3 training images
if len(train_encodings) ==7:
    averaged_encoding = np.mean(train_encodings, axis=0)
else:
    print("Not enough valid training images with detected faces.")
    exit()

# Load and process the verification image
verify_image = download_image(verify_image_url)
verify_face = detect_face(verify_image)

if verify_face is None:
    print("No face detected in the verification image.")
else:
    # Extract encoding for the verification image
    verify_encoding = extract_face_encoding(model, verify_face)

    # Compare the averaged encoding with the verification encoding
    similarity = cosine_similarity([averaged_encoding], [verify_encoding])[0][0]

    # Define a threshold (0.9 is often a good threshold for face matching)
    if similarity > 0.9:
        print("Match: Same person")
    else:
        print("Different people")



1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Match: Same person
